In [61]:
import requests

standings_2024_url = "https://fbref.com/en/comps/9/Premier-League-Stats"

In [62]:
#Get HTML from server
data = requests.get(standings_2024_url)

In [63]:
#Grab all href tags from <a> in the standings table, as we want data from all teams
from bs4 import BeautifulSoup

#Initialize soup object
soup = BeautifulSoup(data.text)

In [64]:
#Select the table element using table.stats_table css selector, specifically select the first table
standings_table = soup.select('table.stats_table')
standings_table


[]

In [66]:
links = standings_table.find_all('a')
links = [l.get("href") for l in links]

#Want to remove all links to do with players and only have links to do with clubs
links = [l for l in links if "/squads" in l] #Include only links with "/squad" inside the link
links #Note that all of these urls are relative paths

AttributeError: ResultSet object has no attribute 'find_all'. You're probably treating a list of elements like a single element. Did you call find_all() when you meant to call find()?

In [48]:
#Add full paths by concatenating string to all
team_urls = [f"https://fbref.com{l}" for l in links]
team_urls


[]

In [49]:
arsenal_url = team_urls[1]
#Get HTML from Arsenal page
arsenal_data = requests.get(arsenal_url)

# arsenal_fixtures = soup.select('table.stats_table')[0]




IndexError: list index out of range

In [50]:
import pandas as pd

#Use HTML to read into pd df, pd will detect tables
#Match parameter - The set of tables containing text matching this regex or string will be returned.
matches = pd.read_html(arsenal_data.text, match="Scores & Fixtures")[0]

ValueError: No tables found

# Extract shooting stats from each match

In [ ]:
soup = BeautifulSoup(arsenal_data.text)
links = soup.find_all("a") #Get all the <a> tags, links at this point includes array full of a tags, we want to get the urls from these a tags (href from a)
links = [l.get("href") for l in links ] #Get all the hrefs
links = [l for l in links if l and 'all_comps/shooting' in l]
links


In [51]:
shooting_html = requests.get(f"https://fbref.com{links[0]}")
shooting_table = pd.read_html(shooting_html.text, match="Shooting")[0]
shooting_table.head() #From head here we can see that there is multi-index level as there are two rows of columns, generally this is not necessary

#We want to be able to index by the column names and so we can drop level
shooting_table.columns = shooting_table.columns.droplevel()
shooting_table.columns

IndexError: list index out of range

In [ ]:
shooting_cols = ["Date", "Sh", "SoT", "SoT%", "G/Sh", "G/SoT", "FK", "PK", "xG"]

match_shape = matches.shape
shooting_shape = shooting_table.shape
print(match_shape, shooting_shape) #(52,19) and (53,26), our dataset is not perfect and for some reason there is a match that is not logged in the table


#With the above in mind when we merge we will only combine the matching dates in both
team_data = matches.merge(shooting_table[shooting_cols], on="Date")
team_data.head()
# print(team_data.shape) #Shape is (52,27)

## Scrape data for multiple teams across multiple seasons

In [52]:
years = list(range(2022,2020, -1))
years

[2022, 2021]

In [53]:
all_matches = []

In [ ]:
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"

In [ ]:
import time

shooting_cols = ["Date", "Sh", "SoT", "SoT%", "G/Sh", "G/SoT", "FK", "PK", "xG"]

for year in years: #Scrape each season
  data = requests.get(standings_url)
  soup = BeautifulSoup(data.text)
  standings_table = soup.select('table.stats_table')[0]

  links = standings_table.find_all('a')
  links = [l.get("href") for l in links]
  links = [l for l in links if "/squads" in l]

  team_urls = [f"https://fbref.com{l}" for l in links]

  for team_url in team_urls: #Scrape each team in a given season
    team_name = team_url.split("/")[-1].replace("-Stats","").replace("-","") #Get the team name
    team_html = requests.get(team_url)
    soup = BeautifulSoup(team_html.text)

    matches = pd.read_html(team_html.text, match="Scores & Fixtures")[0] #Get table of match data

    links = soup.find_all("a")
    links = [l.get("href") for l in links ]
    links = [l for l in links if l and 'all_comps/shooting' in l] #Get shooting links

    shooting_html = requests.get(f"https://fbref.com{links[0]}") #Convert to absolute url
    print(team_name)
    shooting_table = pd.read_html(shooting_html.text, match="Shooting")[0]
    shooting_table.head()
    shooting_table.columns = shooting_table.columns.droplevel()

    try:
      team_data = matches.merge(shooting_table[shooting_cols], on="Date") #Some shooting stats not available for all teams
    except ValueError:
      continue

    team_data[team_data["Comp"] == "Premier League"] #We are predicting premier league so for now we only want data in Premier League

    #Make sure you preserve important data
    team_data["Season"] = year
    team_data["Name"] = team_name

    all_matches.append(team_data)
    time.sleep(1) #Good practice so that you dont scrape too quickly and slow down website


IndexError: list index out of range

In [ ]:
match_df = pd.concat(all_matches)
all_matches

In [ ]:
match_df.to_csv("matches.csv")